In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages")
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [3]:
start_date = "2015-01-01"
end_date = "2022-01-01"

In [3]:
feat_list = ['open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover']
method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]

In [10]:
def generate_alpha_list(feat_list,method,day):
    name_list = []
    # Moving Cov
    if "COV" in method:
        for i in range(len(feat_list)-1):
            for j in range(i+1,len(feat_list)):
                name_list.append("COV_%s_%s_%s" % (feat_list[i],feat_list[j],day))
    if "CORR" in method:
        for i in range(len(feat_list)-1):
            for j in range(i+1,len(feat_list)):
                 name_list.append("CORR_%s_%s_%s" % (feat_list[i],feat_list[j],day))
    if "STD" in method:
        for i in range(len(feat_list)-1):
             name_list.append("STD_%s_%s" % (feat_list[i],day))
    if "ZSCORE" in method:
        for i in range(len(feat_list)-1):
             name_list.append("ZSCORE_%s_%s" % (feat_list[i],day))
    if "RETURN" in method:
        for i in range(len(feat_list)-1):
             name_list.append("RETURN_%s_%s" % (feat_list[i],day))
    if "DECAY" in method:
        for i in range(len(feat_list)-1):
             name_list.append("DECAY_%s_%s" % (feat_list[i],day))
    return name_list

In [11]:
a = generate_alpha_list(feat_list,method,7)
a

['COV_open_high_7',
 'COV_open_low_7',
 'COV_open_close_7',
 'COV_open_vwap_7',
 'COV_open_volume_7',
 'COV_open_pct_chg_7',
 'COV_open_turnover_7',
 'COV_open_free_turnover_7',
 'COV_high_low_7',
 'COV_high_close_7',
 'COV_high_vwap_7',
 'COV_high_volume_7',
 'COV_high_pct_chg_7',
 'COV_high_turnover_7',
 'COV_high_free_turnover_7',
 'COV_low_close_7',
 'COV_low_vwap_7',
 'COV_low_volume_7',
 'COV_low_pct_chg_7',
 'COV_low_turnover_7',
 'COV_low_free_turnover_7',
 'COV_close_vwap_7',
 'COV_close_volume_7',
 'COV_close_pct_chg_7',
 'COV_close_turnover_7',
 'COV_close_free_turnover_7',
 'COV_vwap_volume_7',
 'COV_vwap_pct_chg_7',
 'COV_vwap_turnover_7',
 'COV_vwap_free_turnover_7',
 'COV_volume_pct_chg_7',
 'COV_volume_turnover_7',
 'COV_volume_free_turnover_7',
 'COV_pct_chg_turnover_7',
 'COV_pct_chg_free_turnover_7',
 'COV_turnover_free_turnover_7',
 'CORR_open_high_7',
 'CORR_open_low_7',
 'CORR_open_close_7',
 'CORR_open_vwap_7',
 'CORR_open_volume_7',
 'CORR_open_pct_chg_7',
 'COR

In [28]:
from AlphaNet.Data import concat_original_data


In [29]:
# concat_original_data("test",alpha_list = alpha_list,start_date=start_date,end_date=end_date)
generate_shift_data("test",7,"10d_ret_standscalar")

Loading Feature...: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1705/1705 [00:11<00:00, 154.21it/s]


COV_low_turnover_10_Shift_27  \
timestamp           ticker                                 
2018-02-08 09:30:00 300001                      0.015037   
                    300002                      0.038577   
                    300003                      0.137018   
                    300004                      0.026727   
                    300005                     -0.165676   
...                                                  ...   
2019-12-31 09:30:00 688122                      0.391239   
                    688188                      1.504757   
                    688321                      1.126126   
                    688333                     -0.393946   
                    688388                      0.872699   

                            COV_low_pct_chg_10_Shift_27  \
timestamp           ticker                                
2018-02-08 09:30:00 300001                     0.140560   
                    300002                    -0.312692   
                    300003                     0.891040   
                    300004                     0.100408   
                    300005                     0.057617   
...                                                 ...   
2019-12-31 09:30:00 688122                     0.636114   
                    688188                    -0.264294   
                    688321                    -1.824533   
                    688333                     3.028381   
                    688388                     0.209882   

                            COV_low_volume_10_Shift_27  \
timestamp           ticker                               
2018-02-08 09:30:00 300001                 1308.572029   
                    300002                 5338.529750   
                    300003                19344.497700   
                    300004                 1120.939751   
                    300005               -10661.808391   
...                                                ...   
2019-12-31 09:30:00 688122                 1553.308456   
                    688188                 3447.248349   
                    688321                 4569.743217   
                    688333                 -707.479813   
                    688388                 4611.210739   

                            COV_low_vwap_10_Shift_27  \
timestamp           ticker                             
2018-02-08 09:30:00 300001                  1.240748   
                    300002                  2.286496   
                    300003                  2.387138   
                    300004                  0.228912   
                    300005                  2.396154   
...                                              ...   
2019-12-31 09:30:00 688122                  0.424889   
                    688188                 11.837853   
                    688321                 11.227105   
                    688333                  2.035891   
                    688388                  1.645748   

                            COV_low_close_10_Shift_27  \
timestamp           ticker                              
2018-02-08 09:30:00 300001                   1.176904   
                    300002                   2.419078   
                    300003                   2.237517   
                    300004                   0.178788   
                    300005                   2.369700   
...                                               ...   
2019-12-31 09:30:00 688122                   0.419889   
                    688188                  10.578527   
                    688321                   9.940098   
                    688333                   2.325229   
                    688388                   1.480742   

                            COV_low_turnover_10_Shift_20  \
timestamp           ticker                                 
2018-02-08 09:30:00 300001                     -0.021815   
                    300002                     -0.011051   
             

In [22]:
shift = 3
sequence_list = [0]
for i in range(shift - 1, 30, shift):
    sequence_list.append(i)
sequence_list.sort(reverse=True)
sequence_list

[29, 26, 23, 20, 17, 14, 11, 8, 5, 2, 0]